# Frozen Lake


#### Import all requirements

In [1]:
import gym
import numpy as np
import random
import torch
import time
import matplotlib.pyplot as plt
import pandas as pd


In [20]:
from decimal import Decimal

In [2]:
map1 = ['GASBG']


In [3]:
map2 = ['GBSAG']


In [4]:

env1 = gym.make('FrozenLake-v0', is_slippery=False, desc = map1)

env1.render()


GASBG


In [5]:

env2 = gym.make('FrozenLake-v0', is_slippery=False, desc = map2)

env2.render()


GBSAG


In [6]:
# Total number of States and Actions
n_states = env1.observation_space.n
n_actions = env1.action_space.n
n_rows = 1
n_cols = 5
print( "States = ", n_states)
print( "Actions = ", n_actions)

States =  5
Actions =  4


In [7]:
# Total number of States and Actions
n_states = env2.observation_space.n
n_actions = env2.action_space.n
n_rows = 1
n_cols = 5
print( "States = ", n_states)
print( "Actions = ", n_actions)

States =  5
Actions =  4


In [12]:
def restrict_actions(Q, n_states, n_rows):

  Q.at[n_states -1, :] = np.zeros(n_actions,)
  Q.at[0, :] = np.zeros(n_actions,)
  for i in range( 0, n_states, n_rows): 
    Q.at[i,0] = np.NaN
  for i in range( n_rows -1 , n_states, n_rows): 
    Q.at[i,2] = np.NaN
  for i in range(0, n_rows):
    Q.at[i,3] = np.NaN
  for i in range(n_states - n_rows , n_states):
    Q.at[i,1 ]= np.NaN
  
  return Q


In [39]:
def choose_action(Q, state, epsilon):
  random_for_epsilon = np.random.rand()
  if random_for_epsilon <= epsilon:
    action = random.choice((0,2))
  elif df.loc[state,2] > df.loc[state,0]: 
   action = 2
  elif df.loc[state,0] > df.loc[state,2]: 
    action = 0
  elif df.loc[state,0] == df.loc[state,2]: 
    action = random.choice((0,2))
  return action

In [14]:
##assign index to each state using state-matrix

state_matrix = np.arange(0,n_states).reshape(n_rows,n_cols)
state_matrix

def rowsandcols(state):
  ''' input: state returned by env
      output: location of state as (row,col) tuple'''
  return int(np.where(state_matrix ==state)[0]), int(np.where(state_matrix ==state)[1])

In [15]:
def rewarder1(new_state, reward, entrance_stimulus):

  if entrance_stimulus == 'A':
    if new_state == 0: 
      reward += 100
    else:
      reward -= 10

  if entrance_stimulus == 'B':
    if new_state == 4:
      reward += 100
    else:
      reward -= 10

  return reward

In [16]:
def rewarder2(new_state, reward, entrance_stimulus):

  if entrance_stimulus == 'A':
    if new_state == 4:
      reward += 100
    else:
      reward -= 10

  if entrance_stimulus == 'B':
    if new_state == 0:
      reward += 100
    else:
      reward -= 10

  return reward

In [59]:
reps = 1
num_episodes = 10
steps_total = [] # store number of steps taken in each episode
rewards_total = [] #store reward obtained for each episode
epsilon_total = [] #store epsilon obtained at the end of each episode
terminal_state = []

## q learning

In [65]:
for i in range(reps):
  

  
  epsilon = 0.8
  epsilon_final = 0.1
  epsilon_decay = 0.99

  gamma = 0.90 # discount factor
  learning_rate = 0.9 #how important is the difference between q-val from q-table and what's observed

  Q = pd.DataFrame(np.random.rand(n_states,n_actions)/1000)
  #Q .loc[n_states-1] = np.zeros(n_actions,)
  #Q.loc[n_cols-1] = np.zeros(n_actions,) 
  #Q = restrict_actions(Q, 5, 1)

  for i_episode in range(num_episodes):

    if np.random.randn() > 0.5: 
      entrance_stimulus = 'A'
    else: 
      entrance_stimulus = 'B'

    if epsilon > epsilon_final:
            epsilon *= epsilon_decay


    if i_episode%2 == 0: 

    # resets the environment
      state = env1.reset()
      step = 0
      reward = 0


      while False:
        step += 1

        random_for_epsilon = np.random.randn()

        action = choose_action(Q, state, epsilon)
        

         
        ## env gives reward and next state and whether we've reached terminal state upon taking action at current state.. 
        new_state, _ , done, info = env1.step(action)

        ##if you want reward penalized at for each timestep
        reward = rewarder1(new_state, reward, entrance_stimulus)

        
        # filling the Q Table - 
        
        Q.loc[state, action] = (1- learning_rate)*(Q.at[state, action]) + (learning_rate)*(reward+ gamma*(np.max(Q.loc[new_state, [0,2]])))
        
        # Setting new state for next action
        state = new_state
        env1.render()
        tile = map1[rowsandcols(state)[0]][rowsandcols(state)[1]]
        #if tile == 'G':
          #done =True
        

    
    else: 
      state = env2.reset()
      step = 0
      reward = 0

      while False:
        step += 1

        random_for_epsilon = np.random.randn()
        
        action = choose_action(Q, state, epsilon)
         
        ## env gives reward and next state and whether we've reached terminal state upon taking action at current state.. 
        new_state, _ , done, info = env2.step(action)

        ##if you want reward penalized at for each timestep
        reward = rewarder2(new_state, reward, entrance_stimulus )

        
        # filling the Q Table - 
        
        Q.loc[state, action] = (1- learning_rate)*(Q.loc[state, action]) + (learning_rate)*(reward+ gamma*(np.max(Q.loc[new_state, [0,2] ])))
        
        # Setting new state for next action
        state = new_state
        env2.render()
        tile = map2[rowsandcols(state)[0]][rowsandcols(state)[1]]
        
        #if tile == 'G':
          #done =True


        
      if done:
        
          #print(Q)
          
          #terminal_state.append(tile)
          #steps_total.append(step)
          #rewards_total.append(reward)
          #epsilon_total.append(epsilon)
          #if i_episode % 10 == 0:
        print('Episode: {} Reward: {} Steps Taken: {} Terminal State: {}, Epsilon: {}'.format(i_episode,reward, step, tile, epsilon))
        #break 
  #n_successes.append(terminal_state.count('G'))
  



Episode: 1 Reward: 0 Steps Taken: 0 Terminal State: G, Epsilon: 0.78408
Episode: 3 Reward: 0 Steps Taken: 0 Terminal State: G, Epsilon: 0.768476808
Episode: 5 Reward: 0 Steps Taken: 0 Terminal State: G, Epsilon: 0.7531841195208
Episode: 7 Reward: 0 Steps Taken: 0 Terminal State: G, Epsilon: 0.7381957555423361
Episode: 9 Reward: 0 Steps Taken: 0 Terminal State: G, Epsilon: 0.7235056600070435


In [64]:
env2.reset()

2